<a href="https://colab.research.google.com/github/Amelbnmbh/IBM-DS-Certification/blob/main/Classification_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import the required libraries**


In [ ]:
# Surpress warnings:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix, accuracy_score
import sklearn.metrics as metrics

In [ ]:
!pip install pyodide

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


#Importing the Dataset

In [ ]:
import requests

def download_file(url, filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded: {filename}")
    else:
        print("Failed to download the file.")

In [ ]:
path='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillUp/labs/ML-FinalAssignment/Weather_Data.csv'

In [ ]:
download_file(path, "Weather_Data.csv")

Downloaded: Weather_Data.csv


In [ ]:
df = pd.read_csv("Weather_Data.csv")

In [ ]:
df.head()

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2/1/2008,19.5,22.4,15.6,6.2,0.0,W,41,S,SSW,...,92,84,1017.6,1017.4,8,8,20.7,20.9,Yes,Yes
1,2/2/2008,19.5,25.6,6.0,3.4,2.7,W,41,W,E,...,83,73,1017.9,1016.4,7,7,22.4,24.8,Yes,Yes
2,2/3/2008,21.6,24.5,6.6,2.4,0.1,W,41,ESE,ESE,...,88,86,1016.7,1015.6,7,8,23.5,23.0,Yes,Yes
3,2/4/2008,20.2,22.8,18.8,2.2,0.0,W,41,NNE,E,...,83,90,1014.2,1011.8,8,8,21.4,20.9,Yes,Yes
4,2/5/2008,19.7,25.7,77.4,4.8,0.0,W,41,NNE,W,...,88,74,1008.3,1004.8,8,8,22.5,25.5,Yes,Yes


### Data Preprocessing


#### One Hot Encoding


First, we need to perform one hot encoding to convert categorical variables to binary variables.


In [ ]:
df_sydney_processed = pd.get_dummies(data=df, columns=['RainToday', 'WindGustDir', 'WindDir9am', 'WindDir3pm'])

Next, we replace the values of the 'RainTomorrow' column changing them from a categorical column to a binary column. We do not use the `get_dummies` method because we would end up with two columns for 'RainTomorrow' and we do not want, since 'RainTomorrow' is our target.


In [ ]:
df_sydney_processed.replace(['No', 'Yes'], [0,1], inplace=True)

### Training Data and Test Data


Now, we set our 'features' or x values and our Y or target variable.


In [ ]:
df_sydney_processed.drop('Date',axis=1,inplace=True)

In [ ]:
df_sydney_processed = df_sydney_processed.astype(float)

In [ ]:
features = df_sydney_processed.drop(columns='RainTomorrow', axis=1)
Y = df_sydney_processed['RainTomorrow']

### Linear Regression


#### Q1) Use the `train_test_split` function to split the `features` and `Y` dataframes with a `test_size` of `0.2` and the `random_state` set to `10`.


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, Y, test_size=0.2, random_state=10)

#### Q2) Create and train a Linear Regression model called LinearReg using the training data (`x_train`, `y_train`).


In [ ]:
# Create the Linear Regression model
LinearReg = LinearRegression()

# Train the model using the training data
LinearReg.fit(x_train, y_train)

LinearRegression()

#### Q3) Now use the `predict` method on the testing data (`x_test`) and save it to the array `predictions`.


In [ ]:
predictions = LinearReg.predict(x_test)

#### Q4) Using the `predictions` and the `y_test` dataframe calculate the value for each metric using the appropriate function.


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# Calculate and assign Mean Absolute Error (MAE)
LinearRegression_MAE = mean_absolute_error(y_test, predictions)
# Calculate and assign Mean Squared Error (MSE)
LinearRegression_MSE = mean_squared_error(y_test, predictions)
# Calculate and assign R-squared (R²) Score
LinearRegression_R2 = r2_score(y_test, predictions)

#### Q5) Show the MAE, MSE, and R2 in a tabular format using data frame for the linear model.


In [ ]:
# Create a dictionary with the evaluation metrics
metrics_dict = {
    'Metric': ['MAE', 'MSE', 'R2'],
    'Linear Regression': [LinearRegression_MAE, LinearRegression_MSE, LinearRegression_R2]
}

# Convert the dictionary into a pandas DataFrame
metrics_df = pd.DataFrame(metrics_dict)

# Display the DataFrame
metrics_df

,Metric,Linear Regression
0,MAE,0.256318
1,MSE,0.115721
2,R2,0.427132


### KNN


#### Q6) Create and train a KNN model called KNN using the training data (`x_train`, `y_train`) with the `n_neighbors` parameter set to `4`.


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
# Create the KNN model with n_neighbors set to 4
KNN = KNeighborsRegressor(n_neighbors=4)
# Train the model using the training data
KNN.fit(x_train, y_train)

KNeighborsRegressor(n_neighbors=4)

#### Q7) Now use the `predict` method on the testing data (`x_test`) and save it to the array `predictions`.


In [ ]:
predictions_knn = KNN.predict(x_test)
predictions_knn[:5]

array([0.25, 0.  , 1.  , 0.  , 0.  ])

#### Q8) Using the `predictions` and the `y_test` dataframe calculate the value for each metric using the appropriate function.


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Calculate Mean Absolute Error (MAE)
KNN_MAE = mean_absolute_error(y_test, predictions_knn)

# Calculate Mean Squared Error (MSE)
KNN_MSE = mean_squared_error(y_test, predictions_knn)

# Calculate R-squared (R²) Score
KNN_R2 = r2_score(y_test, predictions_knn)

# Print the results
print("KNN MAE:", KNN_MAE)
print("KNN MSE:", KNN_MSE)
print("KNN R2:", KNN_R2)


KNN MAE: 0.2316793893129771
KNN MSE: 0.1441793893129771
KNN R2: 0.28624847110680307


### Decision Tree


#### Q9) Create and train a Decision Tree model called Tree using the training data (`x_train`, `y_train`).


In [ ]:
from sklearn.tree import DecisionTreeRegressor
# Create the Decision Tree model
Tree = DecisionTreeRegressor()
# Train the model using the training data
Tree.fit(x_train, y_train)

DecisionTreeRegressor()

#### Q10) Now use the `predict` method on the testing data (`x_test`) and save it to the array `predictions`.


In [ ]:
predictions_tree = Tree.predict(x_test)
predictions_tree[:5]

array([0., 0., 1., 0., 0.])

#### Q11) Using the `predictions` and the `y_test` dataframe calculate the value for each metric using the appropriate function.


In [ ]:
from sklearn.metrics import accuracy_score, jaccard_score, f1_score

# Calculate Accuracy
Tree_Accuracy_Score = accuracy_score(y_test, predictions_tree)

# Calculate Jaccard Index (only applicable for binary/multiclass classification)
Tree_JaccardIndex = jaccard_score(y_test, predictions_tree, average='weighted')

# Calculate F1 Score (only applicable for binary/multiclass classification)
Tree_F1_Score = f1_score(y_test, predictions_tree, average='weighted')

# Print the results
print("Decision Tree Accuracy Score:", Tree_Accuracy_Score)
print("Decision Tree Jaccard Index:", Tree_JaccardIndex)
print("Decision Tree F1 Score:", Tree_F1_Score)

Decision Tree Accuracy Score: 0.7587786259541984
Decision Tree Jaccard Index: 0.6251833122895402
Decision Tree F1 Score: 0.7595611837740275


### Logistic Regression



#### Q12) Use the `train_test_split` function to split the `features` and `Y` dataframes with a `test_size` of `0.2` and the `random_state` set to `1`.


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, Y, test_size=0.2, random_state=1)

In [ ]:
print("Training data shape:", x_train.shape, y_train.shape)
print("Testing data shape:", x_test.shape, y_test.shape)

Training data shape: (2616, 66) (2616,)
Testing data shape: (655, 66) (655,)


#### Q13) Create and train a LogisticRegression model called LR using the training data (`x_train`, `y_train`) with the `solver` parameter set to `liblinear`.


In [ ]:
# Create the Logistic Regression model with the solver set to 'liblinear'
LR = LogisticRegression(solver='liblinear')

# Train the model using the training data
LR.fit(x_train, y_train)

LogisticRegression(solver='liblinear')

#### Q14) Now, use the `predict` and `predict_proba` methods on the testing data (`x_test`) and save it as 2 arrays `predictions` and `predict_proba`.


In [ ]:
predictions = LR.predict(x_test)          # Predicted class labels
predict_proba = LR.predict_proba(x_test)  # Predicted probabilities for each class

print("Predictions (class labels):", predictions[:5])  # Display the first 5 predictions
print("Predicted probabilities:\n", predict_proba[:5])  # Display the first 5 predicted probabilities


Predictions (class labels): [0. 0. 0. 0. 0.]
Predicted probabilities:
 [[0.74574813 0.25425187]
 [0.97506424 0.02493576]
 [0.50824637 0.49175363]
 [0.84727479 0.15272521]
 [0.9684321  0.0315679 ]]


#### Q15) Using the `predictions`, `predict_proba` and the `y_test` dataframe calculate the value for each metric using the appropriate function.


In [ ]:
# Calculate Accuracy
LR_Accuracy_Score = accuracy_score(y_test, predictions)

# Calculate Jaccard Index
LR_JaccardIndex = jaccard_score(y_test, predictions)

# Calculate F1 Score
LR_F1_Score = f1_score(y_test, predictions)

# Calculate Log Loss
LR_Log_Loss = log_loss(y_test, predict_proba)

# Print the results
print("Logistic Regression Accuracy Score:", LR_Accuracy_Score)
print("Logistic Regression Jaccard Index:", LR_JaccardIndex)
print("Logistic Regression F1 Score:", LR_F1_Score)
print("Logistic Regression Log Loss:", LR_Log_Loss)

Logistic Regression Accuracy Score: 0.8366412213740458
Logistic Regression Jaccard Index: 0.5091743119266054
Logistic Regression F1 Score: 0.6747720364741642
Logistic Regression Log Loss: 0.3812590636097066


### SVM


#### Q16) Create and train a SVM model called SVM using the training data (`x_train`, `y_train`).


In [ ]:
from sklearn.svm import SVC
# Create the SVM model
SVM = SVC()
# Train the model using the training data
SVM.fit(x_train, y_train)

SVC()

#### Q17) Now use the `predict` method on the testing data (`x_test`) and save it to the array `predictions`.


In [ ]:
predictions = SVM.predict(x_test)
print("SVM Predictions (class labels):", predictions[:5])

SVM Predictions (class labels): [0. 0. 0. 0. 0.]


#### Q18) Using the `predictions` and the `y_test` dataframe calculate the value for each metric using the appropriate function.


In [ ]:
# Calculate Accuracy Score
SVM_Accuracy_Score = accuracy_score(y_test, predictions)

# Calculate Jaccard Index
SVM_JaccardIndex = jaccard_score(y_test, predictions)

# Calculate F1 Score
SVM_F1_Score = f1_score(y_test, predictions)

# Print the results
print("SVM Accuracy Score:", SVM_Accuracy_Score)
print("SVM Jaccard Index:", SVM_JaccardIndex)
print("SVM F1 Score:", SVM_F1_Score)

SVM Accuracy Score: 0.7221374045801526
SVM Jaccard Index: 0.0
SVM F1 Score: 0.0


### Report


#### Q19) Show the Accuracy,Jaccard Index,F1-Score and LogLoss in a tabular format using data frame for all of the above models.

\*LogLoss is only for Logistic Regression Model


In [ ]:
# Create a dictionary to hold the metrics
metrics = {
    'Model': ['Logistic Regression', 'Decision Tree', 'SVM'],
    'Accuracy': [LR_Accuracy_Score, Tree_Accuracy_Score, SVM_Accuracy_Score],
    'Jaccard Index': [LR_JaccardIndex, Tree_JaccardIndex, SVM_JaccardIndex],
    'F1 Score': [LR_F1_Score, Tree_F1_Score, SVM_F1_Score],
    'Log Loss': [LR_Log_Loss, None, None]  # Log Loss is only for Logistic Regression
}

# Create a DataFrame
report = pd.DataFrame(metrics)

# Display the DataFrame
report

,Model,Accuracy,Jaccard Index,F1 Score,Log Loss
0,Logistic Regression,0.836641,0.509174,0.674772,0.381259
1,Decision Tree,0.758779,0.625183,0.759561,NaN
2,SVM,0.722137,0.000000,0.000000,NaN
